## Notebook component to populate a Redshift cluster with our data

In [1]:
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import os

### Access credentials securely

In [2]:
load_dotenv(".env")

user = os.environ.get("REDSHIFT_USERNAME")
password = os.environ.get("REDSHIFT_PASSWORD")
host = os.environ.get("REDSHIFT_HOST")
iamrole = os.environ.get("IAM_role")

### Form connection string

Note, this assumes we added

```bash
poetry add sqlalchemy-redshift

```

In [3]:
url = URL.create(
    drivername="redshift+redshift_connector",
    username=user,
    password=password,
    host=host,
    port=5439,
    database="dev",
)

engine = create_engine(url)

### Load JupySQL extension

In [4]:
%reload_ext sql

Found pyproject.toml from '/Users/macpro/Documents/GitHub/automate-elt-github'

Settings changed:

Config,value
displaycon,False
feedback,True
autopandas,True
named_parameters,True


### Use the engine to initialize access to our Redshift via the alias `redshift`

In [5]:
%sql engine --alias redshift

### Add data to S3 bucket

In [6]:
! aws s3 cp expanded_data/client_account_district.parquet s3://ploomber-redshift-data/nyc-taxi/client_account_district.parquet
! aws s3 cp expanded_data/account_trans_order.parquet s3://ploomber-redshift-data/nyc-taxi/account_trans_order.parquet

upload: expanded_data/client_account_district.parquet to s3://ploomber-redshift-data/nyc-taxi/client_account_district.parquet
upload: expanded_data/account_trans_order.parquet to s3://ploomber-redshift-data/nyc-taxi/account_trans_order.parquet


### Create table `client_account_district` and table `account_trans_order` from data in S3 bucket

Ensure you set up your access key and secret access keys!

In [7]:
%%sql
DROP TABLE IF EXISTS account_trans_order CASCADE;
DROP TABLE IF EXISTS client_account_district CASCADE;

""


In [8]:
%%sql
CREATE TABLE IF NOT EXISTS account_trans_order
(account_id INT,
frequency VARCHAR(50),
account_creation_date INT,
trans_id INT,
transaction_date INT,
transaction_type VARCHAR(50),
operation VARCHAR(50),
transaction_amount INT,
balance INT,
order_id INT,
bank_to VARCHAR(50),
account_to INT,
order_amount INT);

 COPY account_trans_order
FROM 's3://ploomber-redshift-data/nyc-taxi/account_trans_order.parquet'
IAM_ROLE '{{iamrole}}'
FORMAT AS PARQUET;

""


In [9]:
%%sql
CREATE TABLE IF NOT EXISTS client_account_district
(client_id INT,
birth_number INT,
account_id INT,
frequency VARCHAR(50),
account_creation_date INT,
district_name VARCHAR(50),
region VARCHAR(50),
no_of_inhabitants BIGINT,
average_salary BIGINT,
unemployment_rate_95 DOUBLE PRECISION,
unemployment_rate_96 DOUBLE PRECISION,
no_of_entrepreneurs_per_1000_inhabitants INT);

 
COPY client_account_district
FROM 's3://ploomber-redshift-data/nyc-taxi/client_account_district.parquet'
IAM_ROLE '{{iamrole}}'
FORMAT AS PARQUET;

""


### Check for error messages

In [ ]:
result = %sql  SELECT * FROM sys_load_error_detail ORDER BY start_time DESC LIMIT 10;

result['error_message'].to_list()[0:10]

## Profile the data

In [10]:
%%sql
SELECT * FROM client_account_district LIMIT 5

,client_id,birth_number,account_id,frequency,account_creation_date,district_name,region,no_of_inhabitants,average_salary,unemployment_rate_95,unemployment_rate_96,no_of_entrepreneurs_per_1000_inhabitants
0,660,230322,3889,POPLATEK MESICNE,970703,Jindrichuv Hradec,south Bohemia,93931,8427,1.12,1.54,107
1,661,195115,3889,POPLATEK MESICNE,970703,Jindrichuv Hradec,south Bohemia,93931,8427,1.12,1.54,107
2,662,450807,2939,POPLATEK MESICNE,970628,Kolin,central Bohemia,95616,9307,3.85,4.43,118
3,663,230120,2539,POPLATEK MESICNE,970424,Karlovy Vary,west Bohemia,122603,8991,1.39,2.01,128
4,664,805319,4631,POPLATEK PO OBRATU,961212,Vsetin,north Moravia,148545,8909,4.01,5.56,113


In [ ]:
%%sql
SELECT * FROM account_trans_order LIMIT 5

In [11]:
%sqlcmd profile -t client_account_district

,client_id,birth_number,account_id,frequency,account_creation_date,district_name,region,no_of_inhabitants,average_salary,unemployment_rate_95,unemployment_rate_96,no_of_entrepreneurs_per_1000_inhabitants
count,645633,645633,645633,645633,645633,645633,645633,645633,645633,645633,645633,645633
unique,5308,4968,4452,3,1532,76,8,76,75,70,72,44
mean,3385.0000,534775.0000,2894.0000,nan,951531.0000,nan,nan,760299.0000,11146.0000,1.6496,2.0206,143.0000
min,1,110820,1,nan,930101,nan,nan,45714,8110,0.29,0.43,81
max,13998,875927,11382,nan,971229,nan,nan,1204953,12541,7.34,9.4,167


In [12]:
%sqlcmd profile -t account_trans_order

,account_id,frequency,account_creation_date,trans_id,transaction_date,transaction_type,operation,transaction_amount,balance,order_id,bank_to,account_to,order_amount
count,1715140,1715140,1715140,1715140,1715140,1715140,1448066,1715140,1715140,1715140,1715140,1715140,1715140
unique,3758,3,1468,913010,2191,3,5,31786,104530,6471,13,6446,4243
mean,3094.0000,nan,944777.0000,1331562.0000,965826.0000,nan,nan,5635.0000,38774.0000,33974.0000,nan,49244253.0000,3164.0000
min,1,nan,930101,1,930101,nan,nan,0,-35456,29401,nan,399,1
max,11362,nan,971229,3682987,981231,nan,nan,87400,209637,46338,nan,99994199,14882


### Clean up data

In [13]:
! aws s3 rm s3://ploomber-redshift-data/nyc-taxi/client_account_district.parquet
! aws s3 rm s3://ploomber-redshift-data/nyc-taxi/account_trans_order.parquet

### Delete tables

In [ ]:
%%sql
DROP TABLE client_account_district;
DROP TABLE account_trans_order;